# 03-Mini Case Study Data Analysis

## Tujuan Pembelajaran
1. Memahami alur kerja end-to-end analisis data dalam bentuk studi kasus nyata.
2. Menerapkan hasil Exploratory Data Analysis (EDA) ke dalam analisis bisnis.
3. Menarik insight yang relevan dan menyusunnya menjadi rekomendasi berbasis data.
4. Mengkomunikasikan hasil analisis secara terstruktur dan profesional.

## Deskripsi Kasus
Saya berperan sebagai Data Analyst pada sebuah perusahaan e-commerce. Manajemen ingin memahami:
- Pola transaksi pelanggan
- Faktor yang memengaruhi nilai transaksi
- Segmentasi pelanggan sederhana untuk kebutuhan strategi bisnis

Dataset yang digunakan merupakan data transaksi penjualan (simulasi) dengan atribut utama:
- `transaction_id`
- `customer_id`
- `transaction_date`
- `product_category`
- `quantity`
- `price`
- `payment_method`

## Langkah-langkah Pengerjaan
1. Pemahaman Masalah Bisnis
2. Pemahaman Dataset
3. Analisis Data (berbasis EDA Bab 02)
4. Insight & Interpretasi
5. Rekomendasi Bisnis

## 1. Pemahaman Masalah Bisnis
Pertanyaan bisnis utama:
- Kategori produk apa yang paling berkontribusi terhadap revenue?
- Bagaimana pola transaksi pelanggan (frekuensi & nilai transaksi)?
- Metode pembayaran apa yang paling dominan?

**Catatan penting**: Fokus mini case ini bukan pada modeling, tetapi pada analisis dan insight.

## 2. Pemahaman Dataset
### Penjelasan
Dataset diasumsikan sudah melalui tahap data cleaning dasar pada Bab 02.

In [2]:
import pandas as pd

# Membuat dataset dummy e-commerce
data = {
    "order_id": [1001, 1002, 1003, 1004, 1005, 1006],
    "customer_id": ["C001", "C002", "C001", "C003", "C004", "C002"],
    "product_category": ["Elektronik", "Fashion", "Elektronik", "Fashion", "Elektronik", "Fashion"],
    "price": [2500000, 350000, 2500000, 450000, 3200000, 350000],
    "quantity": [1, 2, 1, 1, 1, 3],
    "order_date": pd.to_datetime([
        "2024-01-01", "2024-01-02", "2024-01-05",
        "2024-01-07", "2024-01-08", "2024-01-10"
    ])
}

df = pd.DataFrame(data)
df

,order_id,customer_id,product_category,price,quantity,order_date
0,1001,C001,Elektronik,2500000,1,2024-01-01
1,1002,C002,Fashion,350000,2,2024-01-02
2,1003,C001,Elektronik,2500000,1,2024-01-05
3,1004,C003,Fashion,450000,1,2024-01-07
4,1005,C004,Elektronik,3200000,1,2024-01-08
5,1006,C002,Fashion,350000,3,2024-01-10


In [3]:
# Menyimpan dataset
df.to_csv("data/data_ecommerce.csv", index=False)

In [1]:
# Load Dataset
import pandas as pd

df = pd.read_csv('data/data_ecommerce.csv')
df.head()

,order_id,customer_id,product_category,price,quantity,order_date
0,1001,C001,Elektronik,2500000,1,2024-01-01
1,1002,C002,Fashion,350000,2,2024-01-02
2,1003,C001,Elektronik,2500000,1,2024-01-05
3,1004,C003,Fashion,450000,1,2024-01-07
4,1005,C004,Elektronik,3200000,1,2024-01-08


## 3. Analisis Data
### 3.1 Analisis Revenue
#### Penjelasan
Revenue dihitung dari:
>**Revenue = quantity x price**

In [2]:
# Perhitungan revenue

df['revenue'] = df['quantity'] * df['price']
df[['quantity', 'price', 'revenue']].head()

,quantity,price,revenue
0,1,2500000,2500000
1,2,350000,700000
2,1,2500000,2500000
3,1,450000,450000
4,1,3200000,3200000


In [3]:
# Revenue per Kategori Produk

revenue_category = df.groupby('product_category')['revenue'].sum().sort_values(ascending=False)
revenue_category

product_category
Elektronik    8200000
Fashion       2200000
Name: revenue, dtype: int64

#### Insight
- Kategori dengan revenue tertinggi menunjukkan produk core business.
- Kategori dengan revenue rendah perlu dievaluasi: promosi atau rasionalisasi.

### 3.2 Analisis Perilaku Pelanggan
#### Penjelasan
Analisis dilakukan dengan pendekatan sederhana:
- Total transaksi per pelanggan
- Total revenue per pelanggan

In [5]:
# Aggregasi pelanggan

customer_summary = df.groupby('customer_id').agg(
total_transaction=('order_id', 'count'),
total_revenue=('revenue', 'sum')
).reset_index()

customer_summary.head()

,customer_id,total_transaction,total_revenue
0,C001,2,5000000
1,C002,2,1750000
2,C003,1,450000
3,C004,1,3200000


#### Insight
- Pelanggan dengan transaksi tinggi namun revenue rendah -> low basket size.
- Pelanggan dengan transaksi sedikit namun revenue tinggi -> high value customer.

### 3.3 Analisis Metode Pembayaran

In [9]:
# Menambahkan payment method karena pada dataset sebelumnya tidak ada
df["payment_method"] = [
    "Credit Card",
    "E-Wallet",
    "Credit Card",
    "Bank Transfer",
    "Credit Card",
    "E-Wallet"
]

df.head()

,order_id,customer_id,product_category,price,quantity,order_date,revenue,payment_method
0,1001,C001,Elektronik,2500000,1,2024-01-01,2500000,Credit Card
1,1002,C002,Fashion,350000,2,2024-01-02,700000,E-Wallet
2,1003,C001,Elektronik,2500000,1,2024-01-05,2500000,Credit Card
3,1004,C003,Fashion,450000,1,2024-01-07,450000,Bank Transfer
4,1005,C004,Elektronik,3200000,1,2024-01-08,3200000,Credit Card


In [8]:
# Distribusi Metode Pembayaran

payment_dist = df['payment_method'].value_counts(normalize=True) * 100
payment_dist

payment_method
Credit Card      50.000000
E-Wallet         33.333333
Bank Transfer    16.666667
Name: proportion, dtype: float64

#### Insight
- Metode pembayaran dominan dapat dijadikan fokus kerja sama (bank, e-wallet).
- Metode dengan adopsi rendah perlu edukasi atau insentif.

# 4. Ringkasan
1. **Revenue terbesar** berasal dari sedikit kategori utama (Prinsip Pareto).
2. Terdapat indikasi segmentasi pelanggan berdasarkan nilai transaksi.
3. Preferensi metode pembayaran cukup terkonsentrasi.

Insight ini bersifat `deskriptif-analitis`, bukan asumsi

# 5. Rekomendasi Bisnis
Berdasarkan analisis:
- Fokus promosi pada kategori produk dengan kontribusi revenue tertinggi.
- Terapkan program loyalty untuk pelanggan bernilai tinggi.
- Terapkan program loyalty untuk pelanggan bernilai tinggi.

## Latihan Pengembangan
1. Hitung `rata-rata` revenue per transaksi
2. Tentukan `top 5 customer` berdasarkan total revenue
3. Analisis apakah terdapat perbedaan nilai transaksi antar metode pembayaran.

## Solusi dan Jawaban
### 1. Rata-rata Revenue per Transaksi

In [10]:
# Menghitung rata-rata Revenue per Transaksi

df['revenue'].mean()

1733333.3333333333

### 2. Top 5 Customer

In [11]:
# Melakukan pengecekan 5 teratas customer

customer_summary.sort_values('total_revenue', ascending=False).head(5)

,customer_id,total_transaction,total_revenue
0,C001,2,5000000
3,C004,1,3200000
1,C002,2,1750000
2,C003,1,450000


### 3. Revenue per Metode Pembayaran

In [12]:
# Melakukan pengecekan terhadap rata-rata metode pembayaran

df.groupby('payment_method')['revenue'].mean()

payment_method
Bank Transfer    4.500000e+05
Credit Card      2.733333e+06
E-Wallet         8.750000e+05
Name: revenue, dtype: float64

## Penutup
Mini Case Study ini mencerminkan praktik nyata Data Analyst di industri:
- Tidak selalu membangun model
- Fokus pada business understanding dan data-driven insight

Bab selanjutnya dapat dikembangkan ke:
- Data Visualization
- Customer Segmentation (RFM)
- Predictive Analysis